## 테스트 데이터셋 로드

In [1]:
import os
import urllib.request
import tarfile
import pickle
import numpy as np
from common.ResNet20 import ResNet20
from common.functions import softmax  

def download_cifar100(save_path='cifar-100-python'):
    if os.path.exists(save_path):
        print("CIFAR-100 이미 존재")
        return

    url = 'https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz'
    filename = 'cifar-100-python.tar.gz'
    print("CIFAR-100 다운로드 중...")
    urllib.request.urlretrieve(url, filename)

    with tarfile.open(filename, 'r:gz') as tar:
        tar.extractall()
    os.remove(filename)
    print("다운로드 완료")

def load_batch(filepath):
    with open(filepath, 'rb') as f:
        data_dict = pickle.load(f, encoding='bytes')
    data = data_dict[b'data']
    fine_labels = np.array(data_dict[b'fine_labels'])
    coarse_labels = np.array(data_dict[b'coarse_labels'])
    data = data.reshape(-1, 3, 32, 32).astype(np.float32) / 255.0
    return data, fine_labels, coarse_labels

def load_cifar100_full(data_dir='./cifar-100-python', valid_ratio=0.1):
    # 1. Load train and test batch
    x_train_all, y_train_fine_all, y_train_coarse_all = load_batch(os.path.join(data_dir, 'train'))
    x_test, y_test_fine, y_test_coarse = load_batch(os.path.join(data_dir, 'test'))

    # 2. Split train → train + validation
    num_total = x_train_all.shape[0]
    num_valid = int(num_total * valid_ratio)

    x_valid = x_train_all[:num_valid]
    t_valid_fine = y_train_fine_all[:num_valid]
    t_valid_coarse = y_train_coarse_all[:num_valid]

    x_train = x_train_all[num_valid:]
    t_train_fine = y_train_fine_all[num_valid:]
    t_train_coarse = y_train_coarse_all[num_valid:]

    # 3. Return all
    return (x_train, x_valid, x_test,
            t_train_coarse, t_valid_coarse, y_test_coarse,
            t_train_fine, t_valid_fine, y_test_fine)

In [2]:
# CIFAR-100 다운로드
download_cifar100()

# 전체 데이터셋 로드
(x_train, x_valid, x_test,
 t_train_coarse, t_valid_coarse, t_test_coarse,
 t_train_fine, t_valid_fine, t_test_fine) = load_cifar100_full()

# 확인 출력
print("CIFAR-100 데이터셋 로드 완료:")
print(" - x_train:", x_train.shape)
print(" - x_valid:", x_valid.shape)
print(" - x_test :", x_test.shape)
print(" - t_train_fine :", t_train_fine.shape)
print(" - t_valid_fine :", t_valid_fine.shape)
print(" - t_test_fine  :", t_test_fine.shape)
print(" - t_train_coarse:", t_train_coarse.shape)
print(" - t_valid_coarse:", t_valid_coarse.shape)
print(" - t_test_coarse :", t_test_coarse.shape)


CIFAR-100 이미 존재
CIFAR-100 데이터셋 로드 완료:
 - x_train: (45000, 3, 32, 32)
 - x_valid: (5000, 3, 32, 32)
 - x_test : (10000, 3, 32, 32)
 - t_train_fine : (45000,)
 - t_valid_fine : (5000,)
 - t_test_fine  : (10000,)
 - t_train_coarse: (45000,)
 - t_valid_coarse: (5000,)
 - t_test_coarse : (10000,)


In [3]:
def restore_model_parameters(model, model_state):
    
    model.conv1.W = model_state['conv1_W']
    model.conv1.b = model_state['conv1_b']

    model.fc.W = model_state['fc_W']
    model.fc.b = model_state['fc_b']

    idx = 0
    for block in model.layer1 + model.layer2 + model.layer3:
        for attr in ['conv1', 'conv2', 'shortcut']:
            if hasattr(block, attr):
                conv = getattr(block, attr)
                conv.W = model_state[f'{idx}_W']
                conv.b = model_state[f'{idx}_b']
                idx += 1

def restore_bn_params(model, state):
    bn_count = 0
    for block in model.layer1 + model.layer2 + model.layer3:
        for attr in ['bn1', 'bn2']:
            bn = getattr(block, attr)
            bn.gamma = state[f'{bn_count}_gamma']
            bn.beta = state[f'{bn_count}_beta']
            bn.running_mean = state[f'{bn_count}_running_mean']
            bn.running_var = state[f'{bn_count}_running_var']
            bn_count += 1
        if hasattr(block, 'bn_shortcut'):
            bn = block.bn_shortcut
            bn.gamma = state[f'{bn_count}_gamma']
            bn.beta = state[f'{bn_count}_beta']
            bn.running_mean = state[f'{bn_count}_running_mean']
            bn.running_var = state[f'{bn_count}_running_var']
            bn_count += 1

    bn = model.bn1
    bn.gamma = state[f'{bn_count}_gamma']
    bn.beta = state[f'{bn_count}_beta']
    bn.running_mean = state[f'{bn_count}_running_mean']
    bn.running_var = state[f'{bn_count}_running_var']

In [4]:
def evaluate_model(model, x, y_true):
    batch_size = 100
    preds = []
    for i in range(0, x.shape[0], batch_size):
        x_batch = x[i:i+batch_size]
        logits = model.predict(x_batch)
        probs = softmax(logits)
        y_pred = np.argmax(probs, axis=1)
        preds.append(y_pred)
    preds = np.concatenate(preds)
    acc = np.sum(preds == y_true) / len(y_true)
    return preds, acc

In [5]:
# 모델 평가
model_dir = "final_parameters"
model_files = {
    "crop": os.path.join(model_dir, "crop_epoch_10.pkl"),
    "crop+flip": os.path.join(model_dir, "crop+flip_epoch_10.pkl"),
    "crop+flip+cutout": os.path.join(model_dir, "crop+flip+cutout_epoch_10.pkl")
}

for name, file in model_files.items():
    model = ResNet20()
    with open(file, "rb") as f:
        checkpoint = pickle.load(f)
        model_state = checkpoint["model"]
    restore_bn_params(model, model_state)
    restore_model_parameters(model, model_state)
    _, acc = evaluate_model(model, x_test, t_test_fine)
    print(f"{name} model - [Fine Label] Valid Accuracy: {acc:.4f}")


def predict_softmax(model, x):
    batch_size = 100
    probs = []
    for i in range(0, x.shape[0], batch_size):
        x_batch = x[i:i+batch_size]
        logits = model.predict(x_batch)
        prob = softmax(logits)
        probs.append(prob)
    return np.vstack(probs)
    
# 앙상블 softmax 예측
probs_list = []

for name, file in model_files.items():
    model = ResNet20()
    with open(file, "rb") as f:
        checkpoint = pickle.load(f)
        model_state = checkpoint["model"]
    restore_bn_params(model, model_state)
    restore_model_parameters(model, model_state)
    probs = predict_softmax(model, x_test)
    probs_list.append(probs)



crop model - [Fine Label] Valid Accuracy: 0.4826
crop+flip model - [Fine Label] Valid Accuracy: 0.5061
crop+flip+cutout model - [Fine Label] Valid Accuracy: 0.4944


In [7]:
def get_cifar100_fine_to_coarse_dict():
    """
    CIFAR-100 공식 fine label (0~99) → coarse label (0~19) 매핑 딕셔너리
    """
    fine_to_coarse = {
         4: 0, 30: 0, 55: 0, 72: 0, 95: 0,     # aquatic mammals
         1: 1, 32: 1, 67: 1, 73: 1, 91: 1,     # fish
        54: 2, 62: 2, 70: 2, 82: 2, 92: 2,     # flowers
        9: 3, 10: 3, 16: 3, 28: 3, 61: 3,      # food containers
         0: 4, 51: 4, 53: 4, 57: 4, 83: 4,     # fruit and vegetables
        22: 5, 39: 5, 40: 5, 86: 5, 87: 5,     # household electrical devices
         5: 6, 20: 6, 25: 6, 84: 6, 94: 6,     # household furniture
         6: 7, 7: 7, 14: 7, 18: 7, 24: 7,      # insects
        3: 8, 42: 8, 43: 8, 88: 8, 97: 8,      # large carnivores
        12: 9, 17: 9, 37: 9, 68: 9, 76: 9,     # large man-made outdoor things
        23:10, 33:10, 49:10, 60:10, 71:10,     # large natural outdoor scenes
        15:11, 19:11, 21:11, 31:11, 38:11,     # medium-sized mammals
        34:12, 63:12, 64:12, 66:12, 75:12,     # non-insect invertebrates
        26:13, 45:13, 77:13, 79:13, 99:13,     # people
         2:14, 11:14, 35:14, 46:14, 98:14,     # reptiles
         27:15, 29:15, 44:15, 78:15, 93:15,    # small mammals
         36:16, 50:16, 65:16, 74:16, 80:16,    # trees
         8:17, 13:17, 48:17, 58:17, 90:17,     # vehicles 1
        41:18, 52:18, 56:18, 59:18, 96:18,     # vehicles 2
        47:19, 69:19, 81:19, 85:19, 89:19      # objects
    }
    return fine_to_coarse



def map_softmax_to_coarse(softmax_output, fine_to_coarse):
    """
    softmax_output: shape (N, 100), softmax 확률 결과
    fine_to_coarse: dict, fine label → coarse label 매핑
    
    return: pred_coarse: shape (N,), coarse label 예측 결과
    """
    fine_pred = np.argmax(softmax_output, axis=1)  # shape (N,)
    coarse_pred = np.array([fine_to_coarse[f] for f in fine_pred])  # shape (N,)
    return coarse_pred

In [8]:
# softmax 평균 앙상블
ensemble_probs = np.mean(probs_list, axis=0)
ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_acc = np.sum(ensemble_preds == t_test_fine) / len(t_test_fine)
print(f"[Fine Label] Ensemble Valid Accuracy: {ensemble_acc:.4f}")


#단순 매핑 추가
fine_to_coarse = get_cifar100_fine_to_coarse_dict()
ensemble_preds_coarse = map_softmax_to_coarse(ensemble_probs, fine_to_coarse)
ensemble_coarse_acc = np.mean(ensemble_preds_coarse == t_test_coarse)
print(f"[Coarse Label] Ensemble Accuracy: {ensemble_coarse_acc:.4f}")

[Fine Label] Ensemble Valid Accuracy: 0.5750
[Coarse Label] Ensemble Accuracy: 0.6077


In [9]:
# 가중 평균 앙상블 (Weighted Average Ensemble)
weights = {
    "crop": 0.2,
    "crop+flip": 0.5,
    "crop+flip+cutout": 0.3
}

probs_list = []
for name, file in model_files.items():
    model = ResNet20()
    with open(file, "rb") as f:
        checkpoint = pickle.load(f)
        model_state = checkpoint["model"]
    restore_bn_params(model, model_state)
    restore_model_parameters(model, model_state)
    probs = predict_softmax(model, x_test)
    probs_list.append(weights[name] * probs)

ensemble_probs = np.sum(probs_list, axis=0)
ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_acc = np.sum(ensemble_preds == t_test_fine) / len(t_test_fine)
print(f"[Fine Label] Weighted Ensemble Accuracy: {ensemble_acc:.4f}")


#단순 매핑 추가
fine_to_coarse = get_cifar100_fine_to_coarse_dict()
ensemble_preds_coarse = map_softmax_to_coarse(ensemble_probs, fine_to_coarse)
ensemble_coarse_acc = np.mean(ensemble_preds_coarse == t_test_coarse)
print(f"[Coarse Label] Weighted Ensemble Accuracy: {ensemble_coarse_acc:.4f}")

[Fine Label] Weighted Ensemble Accuracy: 0.5688
[Coarse Label] Weighted Ensemble Accuracy: 0.6017


In [10]:
# 로그 소프트맥스 평균 (Logit Averaging Ensemble)

def predict_logits(model, x):
    batch_size = 100
    logits_all = []
    for i in range(0, x.shape[0], batch_size):
        x_batch = x[i:i+batch_size]
        logits = model.predict(x_batch)
        logits_all.append(logits)
    return np.vstack(logits_all)

logits_list = []
for name, file in model_files.items():
    model = ResNet20()
    with open(file, "rb") as f:
        checkpoint = pickle.load(f)
        model_state = checkpoint["model"]
    restore_bn_params(model, model_state)
    restore_model_parameters(model, model_state)
    logits = predict_logits(model, x_test)
    logits_list.append(logits)

ensemble_logits = np.mean(logits_list, axis=0)
ensemble_probs = softmax(ensemble_logits)
ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_acc = np.sum(ensemble_preds == t_test_fine) / len(t_test_fine)
print(f"[Fine Label] Logit Averaging Ensemble Accuracy: {ensemble_acc:.4f}")

#단순 매핑 추가
fine_to_coarse = get_cifar100_fine_to_coarse_dict()
ensemble_preds_coarse = map_softmax_to_coarse(ensemble_probs, fine_to_coarse)
ensemble_coarse_acc = np.mean(ensemble_preds_coarse == t_test_coarse)
print(f"[Coarse Label] Weighted Ensemble Accuracy: {ensemble_coarse_acc:.4f}")


[Fine Label] Logit Averaging Ensemble Accuracy: 0.5796
[Coarse Label] Weighted Ensemble Accuracy: 0.6160
